In [1]:
import open3d as o3d
import numpy as np


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()


TriangleMesh with 492809 points and 164284 triangles.

In [3]:
mesh

TriangleMesh with 492809 points and 164284 triangles.

In [4]:
def create_coordinate_arrows(size=1.0, origin=[20, 0, 0]):
    """
    좌표축을 화살표로 생성하는 함수
    
    Args:
        size: float - 화살표 크기
        origin: list - 좌표축 원점 위치 [x, y, z]
    
    Returns:
        list: 화살표 geometry 리스트
    """
    geometries = []
    
    # 화살표 생성을 위한 실린더와 원뿔 생성
    for i, color in enumerate([[1,0,0], [0,1,0], [0,0,1]]):  # RGB for XYZ
        # 실린더 (화살표 몸통) 생성
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(
            radius=size/30,
            height=size*0.8
        )
        # 실린더를 중심이 원점에 오도록 이동
        cylinder.translate([0, 0, size*0.4])
        
        # 원뿔 (화살표 머리) 생성
        cone = o3d.geometry.TriangleMesh.create_cone(
            radius=size/15,
            height=size*0.2
        )
        # 원뿔을 실린더 위에 배치
        cone.translate([0, 0, size*0.8])
        
        # 화살표 합치기
        arrow = cylinder + cone
        
        # 각 축 방향으로 회전
        if i == 0:  # X축
            arrow.rotate(
                o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi/2, 0])
            )
        elif i == 1:  # Y축
            arrow.rotate(
                o3d.geometry.get_rotation_matrix_from_xyz([-np.pi/2, 0, 0])
            )
        
        # 원점으로 이동
        arrow.translate(origin)
        
        # 색상 지정
        arrow.paint_uniform_color(color)
        geometries.append(arrow)
    
    return geometries


In [5]:
# 메쉬 크기에 맞춰 좌표축 크기 설정
mesh_bbox = mesh.get_axis_aligned_bounding_box()
mesh_size = mesh_bbox.get_extent()
axis_size = max(mesh_size) * 0.2

# 좌표축 화살표 생성
coordinate_arrows = create_coordinate_arrows(size=axis_size)

# 메쉬와 좌표축 함께 시각화
geometries = [mesh] + coordinate_arrows
o3d.visualization.draw_geometries(geometries)

## 여기까지 모델 + 화살표 그리기

In [6]:
def show(mesh):
    mesh_bbox = mesh.get_axis_aligned_bounding_box()
    mesh_size = mesh_bbox.get_extent()
    axis_size = max(mesh_size) * 0.2

    # 메쉬를 반투명하게 설정
    mesh.paint_uniform_color([0.7, 0.7, 0.7])  # 회색으로 설정
    mesh.vertex_colors = o3d.utility.Vector3dVector(
        np.ones((np.asarray(mesh.vertices).shape[0], 3)) * [0.7, 0.7, 0.7]
    )

    # 좌표축 화살표 생성
    coordinate_arrows = create_coordinate_arrows(size=axis_size)

    # 메쉬와 좌표축 함께 시각화
    geometries = [mesh] + coordinate_arrows
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for g in geometries:
        vis.add_geometry(g)

    opt = vis.get_render_option()
    opt.mesh_show_back_face = True
    
    vis.run()
    vis.destroy_window()

In [7]:
random_angles = [np.random.random()*np.pi*2 for _ in range(3)]
mesh.rotate(o3d.geometry.get_rotation_matrix_from_xyz(random_angles), center=mesh.get_center())

TriangleMesh with 492809 points and 164284 triangles.

In [8]:
show(mesh)

### 자 이제 정렬을 해보자
1. 주성분 분석(PCA)
 - 주성분을 XZ 평면에 수평이 되도록 정렬 (여기서는 그렇슴.)
2. 무게중심이 바운딩중심 과 차이가 날 것임 
 - 앞쪽, 치아 쪽으로 

In [9]:
# 1. PCA 분석
vertices = np.asarray(mesh.vertices)
center = vertices.mean(axis=0)
vertices_centered = vertices - center

In [10]:

covariance_matrix = np.cov(vertices_centered.T)
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

In [11]:
# eigenvalues를 크기순으로 정렬 (가장 큰 값이 마지막에)
idx = eigenvalues.argsort()
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]


In [12]:
lines = []
colors = []
scale_factor = np.max(vertices_centered.max(axis=0) - vertices_centered.min(axis=0)) * 0.5

for i in range(3):
    # 각 주성분 방향으로 라인 생성
    direction = eigenvectors[:, i] * eigenvalues[i] * scale_factor
    line = o3d.geometry.LineSet()
    points = np.array([center - direction, center + direction])
    line.points = o3d.utility.Vector3dVector(points)
    line.lines = o3d.utility.Vector2iVector([[0, 1]])
    
    # RGB 색상 사용 (빨강: 첫번째, 초록: 두번째, 파랑: 세번째 주성분)
    color = np.zeros(3)
    color[i] = 1
    line.colors = o3d.utility.Vector3dVector([color])
    lines.append(line)

print("Eigenvalues (주성분별 분산):", eigenvalues)
print("\nEigenvectors (주성분 방향):\n", eigenvectors)

Eigenvalues (주성분별 분산): [ 27.93309714 191.25079208 305.64572591]

Eigenvectors (주성분 방향):
 [[ 0.09315822 -0.9765462   0.19411095]
 [ 0.17342162 -0.17606377 -0.96898219]
 [ 0.98043179  0.12393169  0.15295244]]


In [13]:
R = np.column_stack([
    eigenvectors[:, 2],  # 첫 번째 주성분 -> x축
    eigenvectors[:, 0],  # 두 번째 주성분 -> y축
    eigenvectors[:, 1]   # 세 번째 주성분 -> z축
])

In [14]:
# 새로운 메쉬 생성 및 회전
aligned_mesh = o3d.geometry.TriangleMesh()
aligned_mesh.vertices = mesh.vertices
aligned_mesh.triangles = mesh.triangles
aligned_mesh.vertex_normals = mesh.vertex_normals
aligned_mesh.compute_vertex_normals()
aligned_mesh.rotate(R.T, center=mesh.get_center())

TriangleMesh with 492809 points and 164284 triangles.

In [15]:
# 원본 메쉬와 주성분 방향 시각화
print("\n원본 메쉬와 주성분 방향:")
show(mesh)


원본 메쉬와 주성분 방향:


In [16]:
print("\n정렬된 메쉬 (첫 번째, 두 번째 주성분이 xy 평면에 수평):")
show(aligned_mesh)


정렬된 메쉬 (첫 번째, 두 번째 주성분이 xy 평면에 수평):


In [17]:
def show(meshes):
    """
    여러 메쉬를 동시에 시각화하는 함수
    
    Args:
        meshes: list of o3d.geometry.TriangleMesh or single mesh
    """
    # 단일 메쉬인 경우 리스트로 변환
    if not isinstance(meshes, list):
        meshes = [meshes]
    
    # 모든 메쉬의 바운딩 박스를 고려하여 좌표축 크기 계산
    max_size = 0
    for mesh in meshes:
        if isinstance(mesh, o3d.geometry.TriangleMesh):
            mesh_bbox = mesh.get_axis_aligned_bounding_box()
            mesh_size = mesh_bbox.get_extent()
            max_size = max(max_size, max(mesh_size))
    
    axis_size = max_size * 0.2

    # 좌표축 화살표 생성
    coordinate_arrows = create_coordinate_arrows(size=axis_size)

    # 모든 메쉬와 좌표축 함께 시각화
    geometries = meshes
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    # 각 geometry 추가
    for g in geometries:
        # 메쉬인 경우 반투명 설정
        if isinstance(g, o3d.geometry.TriangleMesh):
            # 반투명 회색 설정 (알파값 0.5)
            pcd = g.sample_points_uniformly(number_of_points=50000)
            #pcd.paint_uniform_color([0.7, 0.7, 0.7])
            vis.add_geometry(pcd)
        else:
            vis.add_geometry(g)

    for g in coordinate_arrows:
        vis.add_geometry(g)
    # 렌더링 옵션 설정
    opt = vis.get_render_option()
    opt.mesh_show_back_face = True
    opt.mesh_show_wireframe = False
    opt.background_color = np.asarray([0, 0, 0])
    
    # 카메라 설정
    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)
    
    vis.run()
    vis.destroy_window()

In [18]:
show([aligned_mesh, mesh])

In [19]:
obb = aligned_mesh.get_oriented_bounding_box()
obb.color = [1, 0, 0]  # 빨간색

In [20]:
print("OBB 중심점:", obb.center)
print("OBB 크기:", obb.extent)
print("OBB 회전 행렬:\n", obb.R)

OBB 중심점: [-1.56910355 19.4912446   8.24677467]
OBB 크기: [65.11596301 53.82793024 20.16435498]
OBB 회전 행렬:
 [[ 0.96619948 -0.25533398 -0.0355404 ]
 [ 0.02342698 -0.05032802  0.99845794]
 [-0.25672892 -0.96554215 -0.0426452 ]]


In [21]:
# 메쉬와 OBB 함께 시각화
show([aligned_mesh, obb])

In [22]:
center_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=max(obb.extent) * 0.02)
center_sphere.translate(obb.center)
center_sphere.paint_uniform_color([1, 0, 0])  # 빨간색

TriangleMesh with 762 points and 1520 triangles.

In [23]:
mesh_center = aligned_mesh.get_center()
gravity_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=max(obb.extent) * 0.02)
gravity_sphere.translate(mesh_center)
gravity_sphere.paint_uniform_color([0, 0, 1])  # 파란색

TriangleMesh with 762 points and 1520 triangles.

In [24]:
print("OBB 중심점:", obb.center)
print("무게 중심점:", mesh_center)
print("OBB 크기:", obb.extent)
print("OBB 회전 행렬:\n", obb.R)

OBB 중심점: [-1.56910355 19.4912446   8.24677467]
무게 중심점: [-0.14213696 18.35538415  5.7671949 ]
OBB 크기: [65.11596301 53.82793024 20.16435498]
OBB 회전 행렬:
 [[ 0.96619948 -0.25533398 -0.0355404 ]
 [ 0.02342698 -0.05032802  0.99845794]
 [-0.25672892 -0.96554215 -0.0426452 ]]


In [25]:
show([aligned_mesh, obb, center_sphere, gravity_sphere])